Immporting all the required libraraies

In [ ]:
import json  
import pandas as pd  
from pandas.io.json import json_normalize  
import requests
from tabulate import tabulate
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import folium
import warnings
warnings.filterwarnings('ignore')
import os
os.environ.keys()

Getting data from the provided co-ordinates within the range of 10 km

In [ ]:
#Fetching data form HERE API for IIT Bombay
url = 'https://discover.search.hereapi.com/v1/discover?in=circle:17.320486,76.839752;r=10000&q=library&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'
data = requests.get(url).json()
d=json_normalize(data['items'])
d.to_csv('api-data/college.csv')
d 

Now we need to see the availabel data columns first for cleaning of data

In [ ]:
d.columns

Now we will clean data and keep whichever fields are required

In [ ]:
#Cleaning API data
d2=d[['title','address.label','distance','access','position.lat','position.lng','address.postalCode','contacts','id']]
d2.to_csv('api-data/cleaned_apartment.csv')
d2

In [ ]:

df_final=d2[['position.lat','position.lng']]

librayList=[]
latitudes = list(d2['position.lat'])
longitudes = list( d2['position.lng'])
for lat, lng in zip(latitudes, longitudes):    
    radius = '500040' #Set the radius to 1000 metres
    latitude=lat
    longitude=lng
    
	
    search_query = 'libray' #Search for any libray
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    librayList.append(venues['title'].count())



df_final['libray'] = librayList

df_final

In [ ]:
#Run K-means clustering on dataframe
kclusters = 1

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_final)
df_final['Cluster']=kmeans.labels_
df_final['Cluster']=df_final['Cluster'].apply(str)
df_final

In [ ]:
#Plotting clustered locations on map using Folium

#define coordinates of the college
map_bom=folium.Map(location=[17.320486,76.839752],zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
locations = folium.map.FeatureGroup()

# set color scheme for the clusters
def color_producer(cluster):
    if cluster=='0':
        return 'red'
    elif cluster=='1':
        return 'orange'
    else:
        return 'red'

latitudes = list(df_final['position.lat'])
longitudes = list(df_final['position.lng'])
labels = list(df_final['Cluster'])
names=list(d2['title'])
for lat, lng, label,names in zip(latitudes, longitudes, labels,names):
    folium.CircleMarker(
            [lat,lng],
            fill=True,
            fill_opacity=1,
            popup=folium.Popup(names, max_width = 300),
            radius=30,
            color=color_producer(label)
        ).add_to(map_bom)

# add locations to map
map_bom.add_child(locations)
folium.Marker([17.320486,76.839752],popup='Search place',zoom_start=50).add_to(map_bom)

In [ ]:
#saving the map 
map_bom.save("map/libray.html")